In [1]:
from numpy import *
from scipy.sparse import *

In [2]:
trackArtist=load_npz("fucking_ordered_track_artist.npz")
trackAlbum=load_npz("fucking_ordered_track_album.npz")

In [3]:
simByArtist=trackArtist*trackArtist.T
simByAlbum=trackAlbum*trackAlbum.T

In [4]:
simByAlbum.setdiag(0)
simByArtist.setdiag(0)

/home/oppy/.local/lib/python3.6/site-packages/scipy/sparse/compressed.py:742: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  SparseEfficiencyWarning)


In [5]:
simByArtist,simByAlbum

(<100000x100000 sparse matrix of type '<class 'numpy.int64'>'
 	with 2590190 stored elements in Compressed Sparse Row format>,
 <100000x100000 sparse matrix of type '<class 'numpy.int64'>'
 	with 420906 stored elements in Compressed Sparse Row format>)

In [6]:
S=simByArtist+1.5*simByAlbum
S

<100000x100000 sparse matrix of type '<class 'numpy.float64'>'
	with 2492296 stored elements in Compressed Sparse Row format>

In [7]:
R=load_npz("all_playlist_with_tracks_URM.npz")
R

<45649x100000 sparse matrix of type '<class 'numpy.int32'>'
	with 1040522 stored elements in Compressed Sparse Column format>

In [8]:
RR=R*S
RR

<45649x100000 sparse matrix of type '<class 'numpy.float64'>'
	with 20605773 stored elements in Compressed Sparse Column format>

In [9]:
allPlaylist=unique(genfromtxt("train_final.csv",usecols=0,dtype=int32))[1:]

In [10]:
allPlaylist

array([    7569,     7614,     7641, ..., 11765405, 11766293, 11766362], dtype=int32)

In [11]:
allTracks=unique(load("ordered_tracks.npy").astype(int32))
allTracks,allTracks.size

(array([    252,     360,     394, ..., 5007354, 5014702, 5018274], dtype=int32),
 100000)

In [12]:
max([max(x.data) for x in RR if x.data.size!=0])

156.5

In [13]:
RR=RR-200*R
RR

<45649x100000 sparse matrix of type '<class 'numpy.float64'>'
	with 21135411 stored elements in Compressed Sparse Column format>

In [14]:
targetTrack=unique(genfromtxt("target_tracks.csv",dtype=int32,skip_header=1))
targetTrack.size

32195

In [15]:
maskT=list(map(lambda x:x in targetTrack,allTracks))
len(maskT)

100000

In [16]:
targetPlaylist=unique(genfromtxt("target_playlists.csv",skip_header=1,dtype=int32))
targetPlaylist.size

10000

In [17]:
maskP=list(map(lambda x:x in targetPlaylist,allPlaylist))
len(maskP)

45649

In [ ]:
with open("noTagsArtist1Album1,5.csv","w") as f:
    f.write("playlist_id,track_ids\n")
    for i,sims in enumerate(RR):
        if(maskP[i]):
            f.write(str(allPlaylist[i])+",")
            suggestions=allTracks[flip(argsort(sims.multiply(maskT).todense()),1).take(range(5))]
            for t in suggestions[0]:
                f.write(str(t)+" ")
            f.write("\n")